## bodybuilding.com recipe explore

Use beautifulsoup to look at recipes in bb.com database.

Sample page : https://www.bodybuilding.com/recipes/pizza-stuffed-chicken-breast

In [ ]:
import bs4
import pandas as pd
import requests

In [ ]:
# Get the recipe page
sample_recipe_page = requests.get('https://www.bodybuilding.com/recipes/pizza-stuffed-chicken-breast')
sample_recipe = bs4.BeautifulSoup(sample_recipe_page.content, features='html.parser')

In [ ]:
# Get info from page
recipe_name = sample_recipe.find_all('h1', {'class': 'bb-recipe-headline-title'})
nutrients = sample_recipe.find_all('span', {'class': "bb-recipe__meta-nutrient-value"})
nutrient_list = sample_recipe.find_all('span', {'class': "bb-recipe__meta-nutrient-label"})
tags = sample_recipe.find_all('div', {'class': "bb-recipe__topic"})
tag_list = set(x.get_text() for x in tags)
recipe_time = sample_recipe.find_all('time', {'datetime': 'PT60M'})[0].get_text()


In [ ]:
# Get and process ingredients
ingredients_unprocessed = sample_recipe.find_all('li', {'class' : 'bb-recipe__ingredient-list-item'})

ing = [x.get_text() for x in ingredients_unprocessed]

processed_ingredients = [x.split('\r\n ') for x in ing]

new_list = []

for i in processed_ingredients:
    step_1 = [x.strip() for x in i]
    step_2 = [x.split(',')[0] for x in step_1 if x]
    new_list.append(step_2)
    
ingredients_clean = [x[-1] for x in new_list]

print(ingredients_clean)

In [ ]:
# Build a dict for the recipe
recipe_dict = { nutrient_list[x].get_text() : nutrients[x].get_text() for x in range(len(nutrient_list))}
recipe_dict['name'] = recipe_name[0].get_text()
recipe_dict['ingredients'] = [ingredients_clean]
recipe_dict["tags"] = tag_list
recipe_dict["time"] = recipe_time

In [ ]:
recipe_df = pd.DataFrame(recipe_dict, index=[0])

In [ ]:
print(recipe_df)